In [118]:
# !pip install protobuf==3.20.3
# !pip install --upgrade tokenizers
# !pip install crewai crewai_tools langchain_community python-dotenv
# !pip install exifread
# !pip install chardet

In [80]:
# from google.colab import userdata
import os
from dotenv import load_dotenv
load_dotenv()

True

In [81]:
#use the below directories for local runtime
input_dir = "example-files/original-pics/"  # Replace with your input folder
downscaled_dir = "example-files/downscaled-pics/"  # Replace with your output folder
selected_dir = "example-files/selected-pics/"  # Replace with your output folder
example_web_files = "example-files/example-markdown/"
watermark_filepath = "example-files/logo_watermark-youtube.png"
img_filter_threshold = 7.5

In [82]:
from PIL import Image
import imghdr

def resize_images(input_folder, output_folder, max_dimension=1080):
    """
    Resizes images in the input folder, ensuring the largest dimension is max_dimension,
    maintaining aspect ratio, and saves them to the output folder, preserving date taken metadata.

    Args:
      input_folder: Path to the input folder containing images.
      output_folder: Path to the output folder where resized images will be saved.
      max_dimension: The maximum size for the largest dimension (width or height).
    """

    os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        # Check if it's a file before checking image format
        if os.path.isfile(image_path) and imghdr.what(image_path) in ('png', 'jpeg', 'gif', 'bmp'):
            try:
                with Image.open(image_path) as img:
                    # exif_data = img.getexif()  # Get EXIF data
                    # date_taken = exif_data.get(36867) if exif_data else None  # Extract date taken

                    width, height = img.size

                    # Calculate new dimensions based on the largest dimension
                    if width > height:
                        new_width = max_dimension
                        new_height = int(height * (max_dimension / width))
                    else:
                        new_height = max_dimension
                        new_width = int(width * (max_dimension / height))

                    resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)  # Use LANCZOS for better quality

                    # Preserve EXIF data in the resized image
                    # if date_taken:
                    resized_img.info['exif'] = img.info['exif']

                    output_path = os.path.join(output_folder, filename)
                    resized_img.save(output_path, exif=resized_img.info.get('exif'))  # Save with EXIF data
                    print(f"Resized {filename} and saved to {output_folder}")
            except Exception as e:  # Handle potential errors
                print(f"Error processing {filename}: {e}")


resize_images(input_dir, downscaled_dir)

Resized P4060050.jpg and saved to example-files/downscaled-pics/
Resized P4060054.jpg and saved to example-files/downscaled-pics/
Resized P4060213-1.jpg and saved to example-files/downscaled-pics/
Resized P4060269.jpg and saved to example-files/downscaled-pics/
Resized P4060293.jpg and saved to example-files/downscaled-pics/
Resized P4060300.jpg and saved to example-files/downscaled-pics/
Resized P4060396.jpg and saved to example-files/downscaled-pics/
Resized P4060410.jpg and saved to example-files/downscaled-pics/
Resized P4060448.jpg and saved to example-files/downscaled-pics/
Resized P4060478.jpg and saved to example-files/downscaled-pics/
Resized P4060487.jpg and saved to example-files/downscaled-pics/
Resized P4060495.jpg and saved to example-files/downscaled-pics/
Resized P4060506.jpg and saved to example-files/downscaled-pics/
Resized P4060523.jpg and saved to example-files/downscaled-pics/


In [83]:
import base64
import os
import imghdr
import exifread
from pydantic import BaseModel, Field
from typing import List, Optional

class ImageObject(BaseModel):
    filename: str = Field(..., description="Filename of the image.")
    date_taken: Optional[str] = Field(None, description="Date the image was originally captured.")
    base64_encoding: Optional[str] = Field(None, description="Base64 encoded string of the image file.")
    description: Optional[str] = Field(None, description="Objective description of what is in the image.")
    assessment: Optional[str] = Field(None, description="Qualitative assessment of the image.")
    quality_score: Optional[float] = Field(None, ge=1, le=10, description="Quality score of the image based on assessment.")
    title: Optional[str] = Field(None, description="Title of the image.")
    summary: Optional[str] = Field(None, description="Publication ready and web friendly summary of the image.")
    

def images_to_base64_objects(input_folder: str) -> List[ImageObject]:
    """
    Converts images in the input folder to a list of ImageObject objects
    with base64 encoded values and EXIF data.

    Args:
      input_folder: Path to the input folder containing images.

    Returns:
      A list of ImageObject objects with base64 encoded strings and EXIF data.
    """

    image_objects = []

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        # Check if it's a file and a valid image format
        if os.path.isfile(image_path) and imghdr.what(image_path) in ('png', 'jpeg', 'gif', 'bmp'):
            try:
                with open(image_path, "rb") as image_file:
                    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
                    
                    # Extract EXIF data
                    image_file.seek(0)  # Reset file pointer to the beginning
                    tags = exifread.process_file(image_file, stop_tag="EXIF DateTimeOriginal")
                    date_taken = tags.get("EXIF DateTimeOriginal")
                    date_only = date_taken.values.split(" ")[0] if date_taken else None

                    image_object = ImageObject(
                        filename=filename,
                        base64_encoding=encoded_string,
                        date_taken=date_only
                    )
                    image_objects.append(image_object)
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    return image_objects

# Example usage
# downscaled_dir = "path/to/your/downscaled_dir"
image_objects = images_to_base64_objects(downscaled_dir)

In [84]:
# Print the details of each image object
for obj in image_objects:
    print(f"Filename: {obj.filename}")
    print(f"Description: {obj.description}")
    print(f"Assessment: {obj.assessment}")
    print(f"Quality Score: {obj.quality_score}")
    print(f"Date Taken: {obj.date_taken}")
    print(f"Title: {obj.title}")
    print(f"Summary: {obj.summary}")
    print(f"Base64 Encoding (first 40 chars): {obj.base64_encoding[:40] if obj.base64_encoding else 'None'}")
    print("-" * 20)

Filename: P4060448.jpg
Description: None
Assessment: None
Quality Score: None
Date Taken: 2023:04:06
Title: None
Summary: None
Base64 Encoding (first 40 chars): /9j/4AAQSkZJRgABAQAAAQABAAD/4QKURXhpZgAA
--------------------
Filename: P4060478.jpg
Description: None
Assessment: None
Quality Score: None
Date Taken: 2023:04:06
Title: None
Summary: None
Base64 Encoding (first 40 chars): /9j/4AAQSkZJRgABAQAAAQABAAD/4QKURXhpZgAA
--------------------
Filename: P4060487.jpg
Description: None
Assessment: None
Quality Score: None
Date Taken: 2023:04:06
Title: None
Summary: None
Base64 Encoding (first 40 chars): /9j/4AAQSkZJRgABAQAAAQABAAD/4QKURXhpZgAA
--------------------
Filename: P4060495.jpg
Description: None
Assessment: None
Quality Score: None
Date Taken: 2023:04:06
Title: None
Summary: None
Base64 Encoding (first 40 chars): /9j/4AAQSkZJRgABAQAAAQABAAD/4QKURXhpZgAA
--------------------
Filename: P4060506.jpg
Description: None
Assessment: None
Quality Score: None
Date Taken: 2023:04:06
Title: 

In [86]:
from crewai import Agent, Task, Process, Crew, LLM
from crewai_tools import DirectoryReadTool, FileReadTool, FileWriterTool, VisionTool

llm_art_critic = LLM(
    # model="gemini/gemini-1.5-pro",
    model="gemini/gemini-1.5-flash",
    temperature=0.5,
    api_key = os.getenv('GEMINI_API_KEY')
)

In [87]:
agent_critic_background = """You are a highly respected art critic with 20 years of experience in the art market and 30 years
                          as a practicing artist. In your youth, you explored a wide range of artistic mediums,
                          from painting and sculpture to photography and performance art, giving you a deep understanding
                          of the creative process. Your passion for art has led you to travel the world,
                          immersing yourself in diverse cultures and artistic traditions. You possess a profound
                          knowledge of art history, from ancient cave paintings to contemporary installations,
                          and you are constantly seeking new knowledge through lectures, workshops, and exhibitions.
                          Your critical approach is characterized by a keen eye for visual storytelling,
                          an appreciation for the artist's process, and an openness to experimentation.
                          You are a master of articulating your thoughts in a clear and engaging manner,
                          providing insightful critiques that are both objective and constructive."""

art_critic_agent = Agent(
    llm=llm_art_critic,
    role="Art Critic",
    goal="Analyse visual art and write compelling SEO content",
    backstory=agent_critic_background,
    verbose=True,
    memory=True,
    # output_pydantic=ImageAnalysisResult
)

In [95]:

                        # - Your assessment will be based on how good you believe others will find the art work. If
                        #     you yourself like a picture but you think others don't find it striking or interesting
                        #     the image will get a lower score.

agent_critic_task = Task(
    description="""You will provide a detailed and objective analysis of the images given to you as input.
                        Your analysis have to include:
                      1. Objective Description:
                        - Provide a neutral and factual description of the image's subject matter, composition,
                            and key visual elements. Avoid subjective interpretations or emotional responses at this stage.
                        - Pay close attention to the use of color, lines, form, texture, and space.
                      2. Critical Assessment:
                        - Technical Execution: Analyze the artist's technical skill in handling the chosen medium
                            and their mastery of fundamental artistic principles (e.g., perspective, proportion,
                            light and shadow).
                        - Composition and Design: Evaluate the effectiveness of the composition in guiding the
                            viewer's eye and creating visual interest. Consider the use of balance, rhythm,
                            and focal points.
                        - Color Palette: Discuss the artist's use of color, noting the dominant hues, contrasts,
                            and their impact on the overall mood and atmosphere.
                        - Conceptual Exploration: If applicable, delve into the conceptual underpinnings of the work.
                            Identify any symbolism, metaphors, or narratives that contribute to its meaning.
                        - Originality and Innovation: Comment on the originality of the work and whether it
                            demonstrates a unique approach or expands upon existing artistic conventions.¨
                      3. Overall Impression:
                        - Summarize your overall impression of the artwork, highlighting its strengths and weaknesses.
                        - Offer constructive suggestions for improvement, if applicable.
                        - If an image is not good, boring, or poor, you will say so.
                      4. Objective Quality Score:
                        - Assign a score between 1 and 10 (10 being the highest) based on your 
                            assessment of the criteria above.
                        - It is crucial that your score is free from personal bias or preference. Focus on the
                            artwork's technical merits, compositional strength, and conceptual depth.
                        - If an image is not good, boring, or poor, you will say so.
                        - You don't give a good score to boring images. You do give a good score to intriguing images.

                      Remember: Your role is to provide an objective and insightful critique, without aiming to please
                          your assessment is attachedd to your reputation, so you focus on being objective.
                          If an image is not good, boring, or poor, you are not afraid to say so.
                          
                      The image url is provided here, use your VisionTool to view this: {image_path_url}
                      """,
    expected_output="""A list containing the image name, description, critical assessment, and quality score.""",
    agent=art_critic_agent,
    asynchronous=False,
    tools=[VisionTool()]
    # output_pydantic=ImageAnalysisResult #implement later, similar errors https://github.com/crewAIInc/crewAI/discussions/1436
)

# Here is additional information on the image: {crew_image_input}


In [96]:
critique_crew = Crew(
    agents=[art_critic_agent
            ],
    tasks=[
           agent_critic_task,
           ],
    process=Process.sequential,
    verbose=True,
    # memory=True,
    cache = True,
    output_log_file="runtime_agent_critique_log.txt"
)


2025-01-04 14:46:34,335 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [100]:
# image_dictionary
# next(iter(image_dictionary.items()))
# crew_image_input[0]
image_objects[0].base64_encoding
# os.environ['OPENAI_API_KEY'] = os.getenv('GEMINI_API_KEY')


'...'

In [102]:
# results = critique_crew.kickoff(inputs={ "image_url": image_objects[0].base64_encoding})

crew_image_input = { 
    "image_path_url": "C:\D\CREATIVE WIP\Python, Photo Critic\example-files\downscaled-pics\P4060506.jpg"
    }

results = critique_crew.kickoff(inputs=crew_image_input)

# Agent: Art Critic
## Task: You will provide a detailed and objective analysis of the images given to you as input.
Your analysis have to include:
                      1. Objective Description:
- Provide a neutral and factual description of the image's subject matter, composition,
    and key visual elements. Avoid subjective interpretations or emotional responses at this stage.
- Pay close attention to the use of color, lines, form, texture, and space.
                      2. Critical Assessment:
- Technical Execution: Analyze the artist's technical skill in handling the chosen medium
    and their mastery of fundamental artistic principles (e.g., perspective, proportion,
    light and shadow).
- Composition and Design: Evaluate the effectiveness of the composition in guiding the
    viewer's eye and creating visual interest. Consider the use of balance, rhythm,
    and focal points.
- Color Palette: Discuss the artist's use of color, noting the dominant hues, contrasts,
    and th

In [21]:
crew_image_input = []

for image in image_objects:
    # image.base64_encoding = "sverd"
    crew_image_input.append({
        "image_url": image.base64_encoding,
        "crew_image_input": image,
    })

# crew_image_input = image_objects

# crew_image_input[0]

In [22]:
results = critique_crew.kickoff_for_each(inputs=crew_image_input)
# print(results)

2025-01-04 12:39:10,750 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Art Critic
## Task: You will provide a detailed and objective analysis of the images given to you as input.
Your analysis have to include:
                      1. Objective Description:
- Provide a neutral and factual description of the image's subject matter, composition,
    and key visual elements. Avoid subjective interpretations or emotional responses at this stage.
- Pay close attention to the use of color, lines, form, texture, and space.
                      2. Critical Assessment:
- Technical Execution: Analyze the artist's technical skill in handling the chosen medium
    and their mastery of fundamental artistic principles (e.g., perspective, proportion,
    light and shadow).
- Composition and Design: Evaluate the effectiveness of the composition in guiding the
    viewer's eye and creating visual interest. Consider the use of balance, rhythm,
    and focal points.
- Color Palette: Discuss the artist's use of color, noting the dominant hues, contrasts,
    and th

2025-01-04 12:40:48,283 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Art Critic
## Final Answer: 
I cannot complete this task without a valid OpenAI API key.  The Vision Tool requires an API key to function, and I am unable to access and analyze the provided image data.  Please provide a valid API key or an alternative way to access the image.


# Agent: Art Critic
## Task: You will provide a detailed and objective analysis of the images given to you as input.
Your analysis have to include:
                      1. Objective Description:
- Provide a neutral and factual description of the image's subject matter, composition,
    and key visual elements. Avoid subjective interpretations or emotional responses at this stage.
- Pay close attention to the use of color, lines, form, texture, and space.
                      2. Critical Assessment:
- Technical Execution: Analyze the artist's technical skill in handling the chosen medium
    and their mastery of fundamental artistic principles (e.g., perspective, proportion,
    light and shadow).
- 

2025-01-04 12:42:33,860 - 72648 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.InternalServerError: litellm.InternalServerError: VertexAIException - {
  "error": {
    "code": 503,
    "message": "The model is overloaded. Please try again later.",
    "status": "UNAVAILABLE"
  }
}



...



# Agent: Art Critic
## Final Answer: 
I cannot provide a detailed and objective analysis of the image without access to the image itself. The provided base64 encoding is incomplete and unusable by the Vision Tool due to a missing API key.  My analysis would be purely speculative without visual information, and therefore unreliable and unprofessional.  A proper assessment of technical execution, composition, color palette, conceptual exploration, and originality requires visual inspection.  Consequently, I cannot provide an Objective Quality Score.  I apologize for the inadequacy of this response, but the limitations imposed by the lack of an API key and the incomplete image data prevent me from fulfilling the request as intended.


# Agent: Art Critic
## Task: You will provide a detailed and objective analysis of the images given to you as input.
Your analysis have to include:
                      1. Objective Description:
- Provide a neutral and factual description of the image's s

2025-01-04 12:49:38,001 - 72648 - llm.py-llm:170 - ERROR: LiteLLM call failed: litellm.InternalServerError: litellm.InternalServerError: VertexAIException - {
  "error": {
    "code": 503,
    "message": "The model is overloaded. Please try again later.",
    "status": "UNAVAILABLE"
  }
}





LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.

# Agent: Art Critic
## Task: You will provide a detailed and objective analysis of the images given to you as input.
Your analysis have to include:
                      1. Objective Description:
- Provide a neutral and factual description of the image's subject matter, composition,
    and key visual elements. Avoid subjective interpretations or emotional responses at this stage.
- Pay close attention to the use of color, lines, form, texture, and space.
                      2. Critical Assessment:
- Technical Execution: Analyze the artist's technical skill in handling the chosen medium
    and their mastery of fundamental artistic principles (e.g., perspective, proportion,
    light and shadow).
- Composition and Design: Evaluate the effectiveness of the composition in guiding the
    viewer's eye and creating visual interest. Consider the use of balance, rhythm,
    and focal points.
- Color Palette: 

2025-01-04 12:53:04,243 - 72648 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Art Critic
## Final Answer: 
I cannot provide a detailed analysis of the image because the provided `image_path_url` is a base64 encoded string, and I lack the necessary tools to decode and process it.  My analysis requires visual inspection of the image.  The metadata provides the filename ('P4060506.jpg') and date taken ('2023:04:06'), but this is insufficient for a meaningful art critique.  To perform the requested analysis, please provide a publicly accessible URL to the image.


# Agent: Art Critic
## Task: You will provide a detailed and objective analysis of the images given to you as input.
Your analysis have to include:
                      1. Objective Description:
- Provide a neutral and factual description of the image's subject matter, composition,
    and key visual elements. Avoid subjective interpretations or emotional responses at this stage.
- Pay close attention to the use of color, lines, form, texture, and space.
                      2. Critical Assess

In [74]:
import litellm
import json
import time

critic_prompt = agent_critic_background + "\n\n" + agent_critic_task

def analyze_images(image_dictionary):
    """
    Analyzes images in a dictionary, sending them to Gemini for description,
    critical assessment, and quality score.

    Args:
        image_dictionary: A dictionary where keys are image names and values
                           are base64 encoded image strings.

    Returns:
        A dictionary containing analysis results for each image, following
        the defined schema.
    """
    # os.environ['GEMINI_API_KEY'] = userdata.get("GOOGLE_API_KEY")   #use this while in colab

    results = {}

    for image_name, base64_image in image_dictionary.items():
        try:
            response = litellm.completion(
                # model="gemini/gemini-1.5-pro",
                model="gemini/gemini-1.5-flash",
                temperature=0.5,
                max_tokens=4096,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": critic_prompt
                            },
                            {
                                "type": "image_url",
                                "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                            },
                        ],
                    }
                ],
                # api_key=userdata.get("GOOGLE_API_KEY"),  # Replace with your API key source if needed
                response_format={
                    "type": "json_object",
                    "response_schema": {
                        "type": "object",
                        "properties": {
                            "description": {"type": "string"},
                            "critical_assessment": {"type": "string"},
                            "quality_score": {"type": "integer", "minimum": 1, "maximum": 10},
                        },
                        "required": ["description", "critical_assessment", "quality_score"],
                    }
                }
            )

            # Attempt to decode the JSON response
            analysis = json.loads(response.choices[0].message.content) 
            
            # Check if the required keys are present in the analysis
            if all(key in analysis for key in ["description", "critical_assessment", "quality_score"]):
                results[image_name] = analysis
            else:
                print(f"Warning: Incomplete analysis for {image_name}. Missing required keys.")

        except (json.JSONDecodeError, KeyError, IndexError) as e:
            print(f"Error processing {image_name}: {e}")

        except Exception as e:  # Catching other potential exceptions
            print(f"Unexpected error processing {image_name}: {e}")

        time.sleep(1)  # Introduce a delay between API calls. not best practice but works for now

    return results

# Example usage:
# image_dictionary = {"image1.jpg": base64_image_string1, "image2.png": base64_image_string2, ...}
analysis_results = dict()
analysis_results_2 = dict()
analysis_results_3 = dict()
analysis_results = analyze_images(image_dictionary)
analysis_results_2 = analyze_images(image_dictionary)
analysis_results_3 = analyze_images(image_dictionary)
# analysis_results_4 = analyze_images(image_dictionary)
print(analysis_results)

TypeError: can only concatenate str (not "Task") to str

In [119]:
#calculates avg score IF more than one art critique are done

for key, value in list(analysis_results.items()):  # Iterate over a copy of items
    try:
        scores = [
            value.get('quality_score'),  # From analysis_results
            analysis_results_2.get(key, {}).get('quality_score'),
            analysis_results_3.get(key, {}).get('quality_score')
            # analysis_results_4.get(key, {}).get('quality_score')
        ]
        
        if any(score is None or score == 0 for score in scores):
            print(f"Removing {key} due to invalid quality score (None or 0).")
            del analysis_results[key]  # Remove from original dictionary
            if key in analysis_results_2:
                del analysis_results_2[key]
            if key in analysis_results_3:
                del analysis_results_3[key]
            # if key in analysis_results_4:
            #     del analysis_results_4[key]
            continue  # Skip to the next iteration
        
        # If scores are valid, proceed with averaging and updating:
        avg_quality_score = sum(scores) / len(scores)
        analysis_results[key]['quality_score'] = avg_quality_score
        print(analysis_results[key]['quality_score'])
    except KeyError as e:
        print(f"KeyError encountered: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

6.555555555555556
7.555555555555556
6.0
5.777777777777778
4.0
5.0
7.0
5.111111111111111
7.0
8.444444444444445
6.555555555555556
7.0
7.444444444444444
7.0
6.444444444444444
6.0
6.444444444444444
6.0
6.0
6.444444444444444
6.888888888888889
6.0
7.0
6.555555555555556
5.444444444444444
5.111111111111111
5.111111111111111
6.111111111111111
7.555555555555556
6.0
6.666666666666667
7.0
6.111111111111111
7.444444444444444
6.888888888888889
7.0
7.0
7.444444444444444
7.0
5.444444444444444
8.0
6.555555555555556
6.555555555555556
5.111111111111111
6.555555555555556
6.0
6.555555555555556
5.777777777777778
7.0
7.0
7.0
6.555555555555556
7.444444444444444
6.0
6.555555555555556
7.555555555555556
5.555555555555556
7.0
7.333333333333333
7.555555555555556
6.888888888888889
7.0
6.555555555555556
6.555555555555556
6.0
6.888888888888889
6.888888888888889
4.444444444444444
7.444444444444444
4.555555555555556
4.666666666666667
5.555555555555556
6.444444444444444
6.0
6.444444444444444
6.444444444444444
7.0
6.5555

In [120]:
# prompt: save dictionary to txt file, with some formatting or line spaces that make it easy to read

def save_dict_to_txt(data, filename):
  """Saves a dictionary to a text file with formatted output.

  Args:
    data: The dictionary to save.
    filename: The name of the file to save to.
  """
  try:
    with open(filename, 'w') as file:
      for key, value in data.items():
        file.write(f"{key}:\n\n")
        if isinstance(value, dict):
          for k, v in value.items():
            file.write(f"  {k}: {v}\n\n")
        elif isinstance(value, list):
            file.write("  [\n\n")
            for item in value:
                file.write(f"    {item},\n\n")
            file.write("  ]\n\n")
        else:
          file.write(f"  {value}\n\n")
        file.write("\n\n")  # Add an extra newline for better readability

  except Exception as e:
      print(f"An error occurred: {e}")

# Example usage (assuming 'analysis_results' is defined as in your original code)
# Replace with your actual dictionary and file name
save_dict_to_txt(analysis_results, "image_analysis_results.txt")

In [117]:
# implemented changes:
# changing so after avg score is calculated and dict is saved to txt
# every image with quality score over 7.3 is moved to selected_dir
# then i am free to go into the directories and delete images, or add my own
# then filtered_results should be filtered not on 7.3 anymore, 
#     but the dictionary should be filtered to only contain the images that are currently in selected_dir

In [ ]:
# import shutil

def move_high_quality_images(analysis_results, downscaled_dir, selected_dir, threshold):
    """Moves images with a quality score over the threshold to selected_dir.

    Args:
        analysis_results: The dictionary containing analysis results.
        downscaled_dir: The directory containing downscaled images.
        selected_dir: The directory to move selected images to.
        threshold: The quality score threshold for selecting images.
    """
    os.makedirs(selected_dir, exist_ok=True)  # Create the destination directory if it doesn't exist

    for filename, value in analysis_results.items():
        if value.get('quality_score', 0) > threshold:
            source_path = os.path.join(downscaled_dir, filename)
            destination_path = os.path.join(selected_dir, filename)

            if os.path.exists(source_path):
                shutil.move(source_path, destination_path)
                print(f"Moved {filename} to {selected_dir}")
            else:
                print(f"Warning: {filename} not found in {downscaled_dir}")

move_high_quality_images(analysis_results, downscaled_dir, selected_dir, img_filter_threshold)

Moved P4060054.jpg to example-files/selected-pics/
Moved P4060478.jpg to example-files/selected-pics/
Moved P5270188.jpg to example-files/selected-pics/
Moved P6050447.jpg to example-files/selected-pics/
Moved P6220221.jpg to example-files/selected-pics/
Moved P6220471.jpg to example-files/selected-pics/
Moved P8020068-2.jpg to example-files/selected-pics/
Moved P8030142.jpg to example-files/selected-pics/


## if you want to make changes to selected pics, do so here before running the next code block

In [125]:
def filter_results_in_selected_dir(analysis_results, selected_dir):
    """Filters the dictionary to only contain images currently in selected_dir.

    Args:
        analysis_results: The dictionary containing analysis results.
        selected_dir: The directory containing selected images.

    Returns:
        A filtered dictionary containing only images in selected_dir.
    """
    filtered_results = {}
    for filename in os.listdir(selected_dir):
        if filename in analysis_results:
            filtered_results[filename] = analysis_results[filename]
    return filtered_results


filtered_results = filter_results_in_selected_dir(analysis_results, selected_dir)
print(filtered_results)

{'P4060478.jpg': {'critical_assessment': 'The image presents a compelling scene of a lone horseback rider traversing a vast, arid landscape. The technical execution is commendable, with a sharp focus and accurate color representation that captures the subtle variations in the terrain. The composition is strong, utilizing the rule of thirds effectively to place the rider in the lower left, creating a sense of scale and isolation. The warm color palette, dominated by earthy browns and muted oranges, enhances the mood of stillness and solitude. While there is no overt symbolism or narrative, the image evokes a sense of timeless western imagery. The originality lies in the specific composition and the evocative quality of the scene, rather than any radical departure from established photographic conventions. The image is likely to resonate with viewers who appreciate landscape photography and themes of solitude and the American West.', 'description': 'The photograph depicts a lone individu

In [126]:
# prompt: filter the dictionary analysis_results to only keep elements where quality_score is equal to 8 or higher

# filtered_results = {key: value for key, value in analysis_results.items() if value.get('quality_score', 0) >= 7.3}
# filtered_results

In [127]:
# os.makedirs(selected_dir, exist_ok=True)  # Create the destination directory if it doesn't exist

# for filename in filtered_results:  # Iterate through the keys (filenames) in filtered_results
#     source_path = os.path.join(downscaled_dir, filename)
#     destination_path = os.path.join(selected_dir, filename)

#     if os.path.exists(source_path):
#         shutil.move(source_path, destination_path)
#         print(f"Moved {filename} to {selected_dir}")
#     else:
#         print(f"Warning: {filename} not found in {downscaled_dir}")

In [128]:
from pydantic import BaseModel, Field, TypeAdapter
from typing import Dict, Optional, List, Set, Tuple
import os
import exifread

class ImageAnalysisResult(BaseModel):
    filename: str = Field(..., description="Filename of the image.")
    description: str = Field(..., description="Objective description of what is in the image.")
    assessment: str = Field(..., description="Qualitative assessment of the image.")
    quality_score: float = Field(..., ge=1, le=10, description="Quality score of the image based on assessment.")
    date_taken: str = Field(..., description="Date the image was originally captured.")
    title: str = Field(None, description="Title of the image.")
    summary: str = Field(None, description="Publication ready and web friendly summary of the image.")


def convert_to_objects(filtered_results, downscaled_dir):
    image_objects = []
    image_adapter = TypeAdapter(ImageAnalysisResult)  # Create a TypeAdapter
    for filename, analysis in filtered_results.items():
        image_path = os.path.join(downscaled_dir, filename)
        try:
            # Using exifread to get date taken:
            with open(image_path, 'rb') as f:
                tags = exifread.process_file(f)
                date_taken = tags.get('EXIF DateTimeOriginal', tags.get('Image DateTime'))

                # Extract only the date if date_taken is not None:
                if date_taken:
                    date_taken_str = str(date_taken)
                    date_only = date_taken_str.split()[0]  # Split by space and take the first part (date)
                else:
                    date_only = None  # Handle cases where date_taken is None

            # Create ImageAnalysisResult object using TypeAdapter
            image_object = image_adapter.validate_python({
                "filename": filename,
                "description": analysis.get('description', ''),
                "assessment": analysis.get('critical_assessment', ''),
                "quality_score": analysis.get('quality_score', 0),
                "date_taken": date_only  # Use the extracted date
            })
            image_objects.append(image_object)
        except FileNotFoundError:
            print(f"Warning: Image file not found: {image_path}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")
    return image_objects

# Example usage (assuming filtered_results and downscaled_dir are defined)
image_objects = convert_to_objects(filtered_results, input_dir)  # Assuming 'input_dir' is your image directory

# Now you can access the attributes of the objects:
for obj in image_objects:
    print(f"Filename: {obj.filename}")
    print(f"Description: {obj.description}")
    print(f"Assessment: {obj.assessment}")
    print(f"Quality Score: {obj.quality_score}")
    print(f"Date Taken: {obj.date_taken}")
    print(f"Title: {obj.title}")  # Access the generated title
    print(f"Summary: {obj.summary}") # Access the generated summary
    print("-" * 20)

Filename: P4060478.jpg
Description: The photograph depicts a lone individual on horseback, viewed from a distance, moving across a flat, desert-like plain. The rider is small in the frame, emphasizing the vastness of the landscape. The foreground consists of a rocky, uneven terrain with sparse desert vegetation. The mid-ground is the flat plain, extending to the horizon. The background features a range of low, rolling hills and a prominent, reddish-brown butte or mesa that dominates the right side of the image. The sky is a clear, pale blue, indicative of daytime. The overall color palette is muted, with earth tones predominating. The image is well-lit, with natural light illuminating the scene. The composition is horizontal, emphasizing the breadth of the landscape.
Assessment: The image presents a compelling scene of a lone horseback rider traversing a vast, arid landscape. The technical execution is commendable, with a sharp focus and accurate color representation that captures the 

In [129]:
# # prompt: move all images remaining in image_objects from downscaled_dir to selected_dir

# import shutil

# os.makedirs(selected_dir, exist_ok=True)

# for image_object in image_objects:
#     source_path = os.path.join(downscaled_dir, image_object.filename)
#     destination_path = os.path.join(selected_dir, image_object.filename)

#     if os.path.exists(source_path):
#         shutil.move(source_path, destination_path)
#         print(f"Moved {image_object.filename} to {selected_dir}")
#     else:
#         print(f"Warning: {image_object.filename} not found in {downscaled_dir}")

In [130]:
# from PIL import Image

def watermark_images(selected_dir, watermark_filepath):
    watermark = Image.open(watermark_filepath)
    for filename in os.listdir(selected_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(selected_dir, filename)
            img = Image.open(filepath)
            width, height = img.size
            shortest_dim = min(width, height)
            watermark_width = int(0.18 * shortest_dim)
            watermark_height = int((watermark_width / watermark.width) * watermark.height)
            watermark = watermark.resize((watermark_width, watermark_height))
            padding = int(0.01 * shortest_dim)
            img.paste(watermark, (padding, height - watermark_height - padding), watermark)
            img.save(filepath)

watermark_images(selected_dir, watermark_filepath)

In [131]:

def organize_images(selected_dir):
    """
    Creates a new folder for each image in the selected directory,
    moves the image into its corresponding folder, renames it to "featured",
    and creates an index.md file.
    """
    for filename in os.listdir(selected_dir):
        name, ext = os.path.splitext(filename)
        if ext.lower() in ('.jpg', '.jpeg', '.png', '.gif', '.bmp'):
          try:
            # Create a new directory for the image
            new_dir = os.path.join(selected_dir, name)
            os.makedirs(new_dir, exist_ok=True)

            # Move and rename the image
            old_path = os.path.join(selected_dir, filename)
            new_path = os.path.join(new_dir, f"featured{ext}")
            os.rename(old_path, new_path)

            print(f"Moved and renamed '{filename}' to '{new_path}'")

            # Create index.md file
            index_md_path = os.path.join(new_dir, "index.md")
            with open(index_md_path, "w", encoding="utf-8") as f:
                f.write(f"# {name}\n\n")  # Example content, you can customize this
                # f.write("![featured](featured{ext})\n")

            print(f"Created 'index.md' in '{new_dir}'")

          except OSError as e:
              print(f"Error processing {filename}: {e}")

organize_images(selected_dir)

Moved and renamed 'P4060478.jpg' to 'example-files/selected-pics/P4060478\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P4060478'
Moved and renamed 'P4070608.jpg' to 'example-files/selected-pics/P4070608\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P4070608'
Moved and renamed 'P5270188.jpg' to 'example-files/selected-pics/P5270188\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P5270188'
Moved and renamed 'P6040181.jpg' to 'example-files/selected-pics/P6040181\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P6040181'
Moved and renamed 'P6220060.jpg' to 'example-files/selected-pics/P6220060\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P6220060'
Moved and renamed 'P6220221.jpg' to 'example-files/selected-pics/P6220221\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P6220221'
Moved and renamed 'P6220309.jpg' to 'example-files/selected-pics/P6220309\featured.jpg'
Created 'index.md'

In [132]:
subfolder_inputs = []
for subfolder_name in os.listdir(selected_dir):
        # Assuming subfolder names match image filenames without extension
    image_object = next((obj for obj in image_objects if obj.filename.startswith(subfolder_name)), None)
    if image_object:
        #TIS IS WHERE IT STOPS!
        subfolder_inputs.append({
            "subfolder_name": selected_dir + subfolder_name,
            "image_object": image_object  # Pass the relevant image object
        })

subfolder_inputs

[{'subfolder_name': 'example-files/selected-pics/P4060478',
  'image_object': ImageAnalysisResult(filename='P4060478.jpg', description='The photograph depicts a lone individual on horseback, viewed from a distance, moving across a flat, desert-like plain. The rider is small in the frame, emphasizing the vastness of the landscape. The foreground consists of a rocky, uneven terrain with sparse desert vegetation. The mid-ground is the flat plain, extending to the horizon. The background features a range of low, rolling hills and a prominent, reddish-brown butte or mesa that dominates the right side of the image. The sky is a clear, pale blue, indicative of daytime. The overall color palette is muted, with earth tones predominating. The image is well-lit, with natural light illuminating the scene. The composition is horizontal, emphasizing the breadth of the landscape.', assessment='The image presents a compelling scene of a lone horseback rider traversing a vast, arid landscape. The techn

In [232]:
from crewai import Agent, Task, Process, Crew, LLM
from crewai_tools import DirectoryReadTool, FileReadTool, FileWriterTool

# llm_general = LLM(
#     # model="gemini/gemini-1.5-pro",
#     model="gemini/gemini-1.5-flash",
#     temperature=0.4,
#     # max_tokens=4096,
#     # api_key=userdata.get('GOOGLE_API_KEY')    #use when in colab
#     api_key = os.environ['GEMINI_API_KEY']
#     # base_url="https://api.groq.com/openai/v1"
# )

llm_developer = LLM(
    # model="gemini/gemini-1.5-pro",
    model="gemini/gemini-1.5-flash",
    temperature=0.2,
    # max_tokens=4096,
    # api_key=userdata.get('GOOGLE_API_KEY')
    api_key = os.environ['GEMINI_API_KEY']
    # base_url="https://api.groq.com/openai/v1"
)

In [220]:
# from dotenv import load_dotenv
# import os

# load_dotenv(dotenv_path='C:\D\CREATIVE WIP\Python, Photo Critic\.env')

# print(os.getenv('GEMINI_API_KEY'))

In [233]:
critic_full_agent = Agent(
    llm=llm_general,
    role="Art Critic",
    goal="Analyse visual art and write compelling SEO content",
    backstory=agent_critic_background,
    verbose=True,
    memory=True,
    # output_pydantic=ImageAnalysisResult
)

# Define the web developer agent with tools
web_dev_agent = Agent(
    llm=llm_developer,
    role="Web Developer",
    goal="Organize images and create website structure",
    backstory="""You are an experienced web developer proficient in HTML, CSS, Markdown, and JavaScript.
                You are tasked with organizing image files and working on markdown files.
                Use the tools provided to interact with the file system.""",
    verbose=True,
    memory=True
)

In [234]:
read_examples_task = Task(
    description="""Read all the example files from the directory '{example_web_files}'.
                    Store the content of these files in your memory for later use.""",
    expected_output="""Example files read and stored in memory.""",
    agent=web_dev_agent,
    asynchronous=False,
    tools=[DirectoryReadTool(), FileReadTool()]  # Add the necessary tools
)

In [235]:

write_image_web_content = Task(
    description="""You are provided with information and analysis of an image.
                   Each object you receive contains the following information about an image:
                   - filename
                   - description
                   - assessment
                   - quality_score
                   - date_taken
                   - title (currently None)
                   - summary (currently None)

                   Read the content of this list and store it in your memory.
                   For each image object, generate:
                   1. A compelling title based on the description and assessment.
                   2. A compelling summary incorporating key features and artistic qualities.

                   You make sure the title and summary you write are in a similar style,
                   tone, voice, and length as in the example markdown files you have read.

                   The summary is describing the image only, with no reference to the artist himself,
                   as I am the artist and you are making sure the summary is well written on my behalf.
                   The summary should be written to be engaging, without seeming like it was written
                   by generative AI, i.e., use adjectives but do so without being excessive.

                   Make viewers and readers feel engaged and curious about the image, without tooting
                   my own horn too much. E.g., I would never describe my own composition as perfect,
                   I would be more subtle and descriptibe. This needs to be tasteful.

                   Store the generated title and summary within the image object,
                   by updating the 'title' and 'summary' attributes.

                   You do not need to open any files. The necessary information has been provided to 
                   you as task input right here:
                   {current_sub_folder}""",
    expected_output="""A descriptive title, and a summary/description of the image, based on the
                        information you have received about each image.""",
    agent=critic_full_agent,
    asynchronous=False,
    context=[read_examples_task],
    # output_pydantic=ImageAnalysisResult #implement later, similar errors https://github.com/crewAIInc/crewAI/discussions/1436
)

review_image_web_content = Task(
    description="""You perform a review of the each image title and image summary.
                    You make sure the title and summary are written in a similar style,
                   tone, voice, and length as in the example markdown files you have read.

                   The summary is describing the image only, with no reference to the artist himself,
                   as I am the artist and you are making sure the summary is well written on my behalf.
                   The summary should be written to be engaging, without seeming like it was written
                   by generative AI, i.e., use adjectives but do so without being excessive.

                   Make viewers and readers feel engaged and curious about the image, without tooting
                   my own horn too much. E.g., I would never describe my own composition as perfect,
                   I would be more subtle and descriptibe. This needs to be tasteful.

                   You make any changes you deem necessary to improve the text.
                   
                   You make sure the title and summary are based on the task input:
                   {current_sub_folder}""",
    expected_output="""A descriptive title, and a summary/description of the image, based on the
                        information you have received about each image.""",
    agent=critic_full_agent,
    asynchronous=False,
    context=[write_image_web_content],
    # output_pydantic=ImageAnalysisResult #implement later, similar errors https://github.com/crewAIInc/crewAI/discussions/1436
)


In [236]:
# Assuming image_objects and selected_dir are defined as in your original code

# subfolder_inputs = []
# for subfolder_name in os.listdir(selected_dir):
#     # Assuming subfolder names match image filenames without extension
#     image_object = next((obj for obj in image_objects if obj.filename.startswith(subfolder_name)), None)
#     if image_object:
#         subfolder_inputs.append({
#             "subfolder_name": subfolder_name,
#             "image_object": image_object  # Pass the relevant image object
#         })

In [237]:

process_all_subfolders_task = Task(
    description="""Process a single subfolder using provided input data
                   Navigate to the directory named in '{current_sub_folder}'.
                   Perform the processes:
                       1. Get the name of the current subfolder.
                       2. Find the corresponding image object in the subfolder data list
                          by matching the subfolder name with the object's 'filename' attribute (without extension).
                       3. Open the file named 'index.md' in the subfolder. If it doesn't exist, create it.
                       4. Based on the example web markdown files you read earlier and the image object's data,
                          populate the 'index.md' file using the following attributes from the image objects:
                           - Title: Use the 'title' attribute from the image object.
                           - Date: Use the 'date_taken' attribute from the image object.
                           - Image Summary: Use the 'summary' attribute from the image object.
                   Use DirectoryReadTool to list files, FileReadTool to read files, and FileWriterTool to update/create the 'index.md' files.
                   It is important that you do not rewrite the image summary or the image title. You should use the
                   summary and the title you received from the Art Critic Agent""",
    expected_output="""'index.md' files updated/created with content from image objects.
                    AND SAVED IN THE SUBFOLDER IN: '{current_sub_folder}'.
                    Characters aligned with UTF-8 standards and best practices.
                    the file must include the attributes:
                    "title: "
                    "authors:
                      -admin"
                    "tags:
                      - Photography"
                    "image:
                      placement: 2
                      preview_only: false"
                    """,
    agent=web_dev_agent,
    context=[read_examples_task, review_image_web_content],
    tools=[DirectoryReadTool(), FileReadTool(), FileWriterTool()],  # Add the necessary tools
    asynchronous=False,
)


In [238]:
# removing some already written folders 
# subfolder_inputs = [item for item in subfolder_inputs if item['subfolder_name'] != 'P4100073']


In [239]:
full_crew = Crew(
    agents=[critic_full_agent,
            web_dev_agent
            ],
    tasks=[
           read_examples_task,
           write_image_web_content,
           review_image_web_content,
           process_all_subfolders_task,
          #  review_index_structure
           ],
    process=Process.sequential,
    verbose=True,
    # memory=True,
    cache = True,
    output_log_file="runtime_agent_log.txt"
)


2025-01-03 21:57:14,871 - 27408 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [240]:
# consider adding task guardrails to verify output

In [241]:
# crew_inputs = {
#     "example_web_files": example_web_files,
#     "current_sub_folder": subfolder_inputs[0]
# }

# results = full_crew.kickoff(inputs=crew_inputs)
# # print(results)

In [242]:
crew_inputs = []

for sub_folder in subfolder_inputs:
    crew_inputs.append({
        "current_sub_folder": sub_folder,
        "example_web_files": example_web_files
    })

results = full_crew.kickoff_for_each(inputs=crew_inputs)
# print(results)

2025-01-03 21:57:19,034 - 27408 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Web Developer
## Task: Read all the example files from the directory 'example-files/example-markdown/'.
                    Store the content of these files in your memory for later use.


# Agent: Web Developer
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"example-files/example-markdown/\"}"
## Tool Output: 
File paths: 
-example-files/example-markdown/index1.md
- example-files/example-markdown/index2.md
- example-files/example-markdown/index3.md


# Agent: Web Developer
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"example-files/example-markdown/index1.md\"}"
## Tool Output: 
---
title: 'Roadrunner in Big Bend National Park'
authors:
- admin
tags:
- Photography
date: "2023-04-06T00:00:00Z"
featured: false
draft: false

image:
  placement: 2
  focal_point: ""
  preview_only: false
---

A Greater Roadrunner stands alert on a rocky outcrop in Big Bend National Park, Texas. The rugged mountains provide a stunning backdrop to 

2025-01-03 21:59:07,316 - 27408 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Web Developer
## Final Answer: 
---\ntitle: "Desert Solitude"\nauthors:\n  -admin\ntags:\n  - Photography\ndate: "2023-04-06T00:00:00Z"\nfeatured: false\ndraft: false\n\nimage:\n  placement: 2\n  preview_only: false\n---\n\nA breathtaking vista unfolds: a lone horseback rider, a tiny figure against the immensity of a desert plain stretching to the horizon.  The muted palette of browns and oranges, punctuated by a striking reddish-brown mesa, evokes a sense of quiet contemplation.  Sparse desert vegetation and rocky terrain in the foreground contrast with the smooth expanse of the mid-ground, creating a powerful sense of scale and isolation. The clear pale blue sky completes this evocative landscape.


# Agent: Web Developer
## Task: Read all the example files from the directory 'example-files/example-markdown/'.
                    Store the content of these files in your memory for later use.


# Agent: Web Developer
## Using tool: List files in directory
## Tool Input: 
"{

2025-01-03 22:02:01,903 - 27408 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


...



# Agent: Web Developer
## Final Answer: 
"---\ntitle: \"Vintage Ford Pickup: A Study in Rust and Nostalgia\"\nauthors:\n  - admin\ntags:\n  - Photography\ndate: \"2024-06-23T00:00:00Z\"\nfeatured: false\ndraft: false\n\nimage:\n  placement: 2\n  preview_only: false\n---\n\nBathed in sunlight, a vintage Ford pickup truck stands as a poignant symbol of time's passage. Its once off-white paint is now a tapestry of rust, each imperfection telling a story of journeys past.  The truck's weathered charm is sharply contrasted by the sleek lines of a modern building in the background, while a diagonal section of chain-link fence adds a touch of urban grit. The detail is striking; you can almost feel the texture of the aged metal under your fingertips. This image whispers tales of bygone eras, inviting viewers to contemplate the beauty of decay and the enduring spirit of classic Americana."


# Agent: Web Developer
## Task: Read all the example files from the directory 'example-files/example-m

2025-01-03 22:11:43,072 - 27408 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Web Developer
## Final Answer: 
"---\ntitle: \"Rustic Hen Portrait\"\nauthors:\n  -admin\ntags:\n  - Photography\ndate: \"2022-07-23T00:00:00Z\"\nfeatured: false\ndraft: false\n\nimage:\n  placement: 2\n  preview_only: false\n---\n\nA captivating close-up reveals the intricate details of a hen's plumage.  Light beige feathers, speckled with brown, are sharply focused, creating a textural richness. The vibrant red of the comb and wattles offer a striking contrast against the muted background, subtly blurred to emphasize the hen.  Natural light enhances the scene, casting delicate shadows that add depth and dimension to the image.  The overall impression is one of quiet strength and natural beauty."


# Agent: Web Developer
## Task: Read all the example files from the directory 'example-files/example-markdown/'.
                    Store the content of these files in your memory for later use.


# Agent: Web Developer
## Using tool: List files in directory
## Tool Input: 
"{\"

2025-01-03 22:14:27,332 - 27408 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Web Developer
## Final Answer: 
"---\ntitle: \"Austin Skyline: A Sun-Drenched Perspective from a City Bridge\"\nauthors:\n  -admin\ntags:\n  - Photography\ndate: \"2022-08-02T00:00:00Z\"\nfeatured: false\ndraft: false\n\nimage:\n  placement: 2\n  preview_only: false\n---\n\nExperience the Austin skyline from a unique vantage point: a sun-drenched pedestrian bridge.  The photograph's composition masterfully utilizes the bridge's metallic railing and its shadow to guide the viewer's gaze towards the iconic Texas State Capitol in the distance.  Modern architecture punctuates the bright cityscape, creating a compelling interplay of light and shadow.  The overall effect is a vibrant and balanced image, capturing the energy of Austin on a clear day."


# Agent: Web Developer
## Task: Read all the example files from the directory 'example-files/example-markdown/'.
                    Store the content of these files in your memory for later use.


# Agent: Web Developer
## Using to

In [144]:
import chardet

def ensure_utf8_encoding(selected_dir):
    """
    Recursively processes all Markdown files in the selected directory and
    its subdirectories, ensuring they are encoded in UTF-8.
    
    Args:
        selected_dir (str): Path to the folder containing Markdown files.
    """
    for root, dirs, files in os.walk(selected_dir):
        for file in files:
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                # Read the file and detect its current encoding
                with open(file_path, 'rb') as f:
                    raw_data = f.read()
                    detected = chardet.detect(raw_data)
                    current_encoding = detected['encoding']
                
                # If not UTF-8, convert and overwrite
                if current_encoding.lower() != 'utf-8':
                    with open(file_path, 'w', encoding='utf-8') as f:
                        f.write(raw_data.decode(current_encoding))
                    print(f"Converted {file_path} to UTF-8.")
                else:
                    print(f"{file_path} is already UTF-8.")

# Example usage
ensure_utf8_encoding(selected_dir)

Converted example-files/selected-pics/P4060478\index.md to UTF-8.
Converted example-files/selected-pics/P4070608\index.md to UTF-8.
Converted example-files/selected-pics/P5270188\index.md to UTF-8.
Converted example-files/selected-pics/P6040181\index.md to UTF-8.
Converted example-files/selected-pics/P6220060\index.md to UTF-8.
Converted example-files/selected-pics/P6220221\index.md to UTF-8.
Converted example-files/selected-pics/P6220309\index.md to UTF-8.
Converted example-files/selected-pics/P6230571\index.md to UTF-8.
Converted example-files/selected-pics/P6230622\index.md to UTF-8.
Converted example-files/selected-pics/P7230316\index.md to UTF-8.
Converted example-files/selected-pics/P8020068-2\index.md to UTF-8.
Converted example-files/selected-pics/P8090407\index.md to UTF-8.


### code graveyard for reference

In [ ]:
# review_index_structure = Task(
#     description=f"""You open the folder '{selected_dir}'.
#                     In this folder there are multiple sub folders, each named after an image.
#                     You iterate through each sub folder. You read the index.md files in each subfolder
#                     and compare the file to the example markdown files you previously read
#                     in the task 'read_examples_task'.

#                     You verify that the technical structure of the index.md files is correct,
#                     and the technical structure is identical to the example index markdown files.

#                     This includes verifying the correct variables, date structure, and similar.
#                     You DO NOT change any of the content itself, which is the title and the image description.
#                     """,
#     expected_output=f"""'index.md' files updated in all subfolders of '{selected_dir}' with content from image objects.
#                     The title and the image description/summary will NOT be changed by you.
#                     The file must include the attributes:
#                     "title: "
#                     "authors:
#                       -admin"
#                     "tags:
#                       - Photography"
#                     "image:
#                       placement: 2
#                       preview_only: false"
#                     """,
#     agent=web_dev_agent,
#     asynchronous=False,
#     context=[read_examples_task, process_all_subfolders_task],
#     tools=[DirectoryReadTool(), FileReadTool(), FileWriterTool()],
#     input_data={
#         "selected_dir": selected_dir
#     }
# )